In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from os import system
from IPython.display import clear_output
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import phik 
import seaborn as sns
from phik import resources, report
from phik.report import plot_correlation_matrix
from itertools import combinations
import random
from tabulate import tabulate
from IPython.display import display, HTML

/public/apps/python/3.7.0/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# dividing big function into smaller functions

In [2]:
def gtex_data_processing(gtex_gct_dataset):
    gtex_breast_data_all=pd.read_csv(gtex_gct_dataset,sep='\t')
    gtex_breast_data_all=gtex_breast_data_all.T.copy()
    gtex_breast_data_all.columns=gtex_breast_data_all.iloc[2]
    df_int=gtex_breast_data_all.loc[(gtex_breast_data_all.index!='id')&(gtex_breast_data_all.index!='Name')&(gtex_breast_data_all.index!='Description')].astype('float').copy()
    return df_int

def tcga_data_processing(tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info):
    tcga_brca_df = pd.read_csv(tcga_hiseq_dataset, sep = '\t', index_col=0)
    tcga_brca_sample_info = pd.read_csv(tcga_hiseq_corresponding_sample_info,sep="\t",index_col=0)
    #drop metastatic from main sample type data frame
    tcga_brca_sample_info = tcga_brca_sample_info[~(tcga_brca_sample_info.sample_type=='Metastatic')]

    #separate normal and primtum
    tcga_brca_normal_patients = tcga_brca_sample_info[tcga_brca_sample_info.sample_type=='Solid Tissue Normal']
    tcga_brca_primtum_patients = tcga_brca_sample_info[tcga_brca_sample_info.sample_type=='Primary Tumor']

    #drop index of samples not in tcga_brca_df
    tcga_brca_normal_patients=  tcga_brca_normal_patients.drop(non_match for non_match in tcga_brca_normal_patients.index.values if non_match not in tcga_brca_df.columns)
    tcga_brca_primtum_patients =tcga_brca_primtum_patients.drop(non_match for non_match in tcga_brca_primtum_patients.index.values if non_match not in tcga_brca_df.columns) 

    #make sure both normal and tumor patients match, and drop patients not in tcga_brca_df
    matched_patient_index_list=[]
    for patient in tcga_brca_primtum_patients.patient_id.values:
        if patient in tcga_brca_normal_patients.patient_id.values and patient in [patient_id.split('-')[2] for patient_id in tcga_brca_df.columns]:
            matched_patient_index_list.append(tcga_brca_sample_info[tcga_brca_sample_info.patient_id==patient].index.values)
    matched_patient_list=tcga_brca_sample_info.loc[[x for l in matched_patient_index_list for x in l]].copy()


    #separate normal and primary tumor
    normal_matched_patient_list=matched_patient_list[matched_patient_list.sample_type=='Solid Tissue Normal']
    primtum_matched_patient_list = matched_patient_list[matched_patient_list.sample_type=='Primary Tumor']

    #matched normal and primary tumor samples
    tcga_brca_df_normal_matched=tcga_brca_df[tcga_brca_df.columns[tcga_brca_df.columns.isin(normal_matched_patient_list.index.values)]]
    tcga_brca_df_primtum_matched=tcga_brca_df[tcga_brca_df.columns[tcga_brca_df.columns.isin(primtum_matched_patient_list.index.values)]]

    #test to see if normal and primary tumor patient IDs are a match
    for i in [patient_id.split('-')[2] for patient_id in tcga_brca_df_normal_matched.columns]:
        if i not in [patient_id.split('-')[2] for patient_id in tcga_brca_df_primtum_matched.columns]:
            print('trouble')
            
    #TCGA brca normal ALL
    tcga_brca_df_normal_all = tcga_brca_df.T.loc[tcga_brca_normal_patients.index.values].copy()
    # dataframe with all TCGA primary tumor samples (not necessarily matched)
    tcga_brca_df_primtum_all= tcga_brca_df.T.loc[tcga_brca_primtum_patients.index.values].copy()
    
    return {'tcga_brca_df_normal_matched':tcga_brca_df_normal_matched,
           'tcga_brca_df_primtum_matched':tcga_brca_df_primtum_matched,
           'tcga_brca_df_normal_all':tcga_brca_df_normal_all,
           'tcga_brca_df_primtum_all':tcga_brca_df_primtum_all}

    

In [3]:
def correlation_selected_gene(provided_dataset,interest_gene,correlation_method):
    dataset_corr = provided_dataset.corrwith(provided_dataset[interest_gene],method=correlation_method)
    dataset_corr = dataset_corr.dropna(axis=0,how='any')
    idx = dataset_corr.index.drop_duplicates(keep=False)
    dataset_corr = dataset_corr.loc[idx]
    return dataset_corr

def count_high_corr_genes_no_overlap(interest_gene,correlation_method,correlation_cutoff,*processed_datasets):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    print('no overlap between datasets')
    dict_empty={}
    j=0
    for dataset in processed_datasets:
        dataset=correlation_selected_gene(dataset,interest_gene,correlation_method)
        dict_empty[input(f'insert name for {j+1}th dataset')]=dataset[dataset>=correlation_cutoff].shape[0]
        
        j+=1
    
    dict_pd_df = pd.DataFrame(dict_empty,index=[f'HIGHLY_correlated_with_{interest_gene}'])
    #return display(HTML(dict_pd_df.to_html()))
    return dict_pd_df

def count_high_corr_genes_with_overlap(interest_gene,correlation_method,correlation_cutoff,dataset1,dataset2):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    dataset1=correlation_selected_gene(dataset1,interest_gene,correlation_method)
    dataset2=correlation_selected_gene(dataset2,interest_gene,correlation_method)
    common_gene_list= dataset1.index.intersection(dataset2.index)
    print("1st: overlapping genes identified\n2nd: each dataset subset with overlapping genes\n3rd: cutoff applied")
    dict_empty={}
    j=0
    for dataset in [dataset1,dataset2]:
        dataset=dataset[common_gene_list]
        dict_empty[input(f'insert name for {j+1}th dataset')]=dataset[dataset>=correlation_cutoff].shape[0]
        j+=1
    dict_pd_df = pd.DataFrame(dict_empty,index=[f'HIGHLY_correlated_with_{interest_gene}'])
    
    return display(HTML(dict_pd_df.to_html()))

def cutoff_applied_togtex_2_high_both(interest_gene,correlation_method,correlation_cutoff,gtex_dataset,other_dataset):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    gtex_dataset=correlation_selected_gene(gtex_dataset,interest_gene,correlation_method)
    gtex_dataset = gtex_dataset[gtex_dataset>=correlation_cutoff]
    dataset2=correlation_selected_gene(other_dataset,interest_gene,correlation_method)
    common_gene_list= gtex_dataset.index.intersection(dataset2.index)
    global new_dataset    
    new_dataset = pd.concat([gtex_dataset[common_gene_list],dataset2[common_gene_list]],axis=1)
    first_dataset_name = 'gtex_cutoffmet'
    other_dataset_name=input('insert name for other dataset')
    new_dataset.columns=[first_dataset_name,other_dataset_name]
    print("1st: high cutoff applied to gtex dataset\n2nd: common genes identified with other dataset\n3rd: both datasets combined\n4th: High cutoff applied to non-gtex dataset")
    
    new_dataset = new_dataset[new_dataset[other_dataset_name]>=correlation_cutoff]
    dict_pd_df = pd.DataFrame(data=[[new_dataset.shape[0],new_dataset.shape[0]]],
                             index=[f'high_corr_with_{interest_gene}_both_same_genes'],
                             columns=[first_dataset_name,other_dataset_name])
    return dict_pd_df

def cutoff_applied_togtex_2_high_low(interest_gene,correlation_method,correlation_cutoff,opposing_corr_cutoff,gtex_dataset,other_dataset):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    gtex_dataset=correlation_selected_gene(gtex_dataset,interest_gene,correlation_method)
    gtex_dataset = gtex_dataset[gtex_dataset>=correlation_cutoff]
    dataset2=correlation_selected_gene(other_dataset,interest_gene,correlation_method)
    common_gene_list= gtex_dataset.index.intersection(dataset2.index)
    global new_dataset    
    new_dataset = pd.concat([gtex_dataset[common_gene_list],dataset2[common_gene_list]],axis=1)
    first_dataset_name = 'gtex_cutoffmet'
    other_dataset_name=input('insert name for other dataset')
    new_dataset.columns=[first_dataset_name,other_dataset_name]
    print("1st: high cutoff applied to gtex dataset\n2nd: common genes identified with other dataset\n3rd: both datasets combined\n4th: low cutoff applied to non-gtex dataset")
    
    new_dataset = new_dataset[new_dataset[other_dataset_name]<=opposing_corr_cutoff]
    dict_pd_df = pd.DataFrame(data=[[new_dataset.shape[0],new_dataset.shape[0]]],
                             index=[f'corr_with_{interest_gene}_highgtex_low_other'],
                             columns=[first_dataset_name,other_dataset_name])
    return display(HTML(dict_pd_df.to_html()))
    
def cutoff_applied_totcga_2_high_both(interest_gene,correlation_method,correlation_cutoff,gtex_dataset,other_dataset):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    gtex_dataset=correlation_selected_gene(gtex_dataset,interest_gene,correlation_method)
    dataset2=correlation_selected_gene(other_dataset,interest_gene,correlation_method)
    dataset2 = dataset2[dataset2>=correlation_cutoff]
    common_gene_list= gtex_dataset.index.intersection(dataset2.index)
    global new_dataset    
    new_dataset = pd.concat([gtex_dataset[common_gene_list],dataset2[common_gene_list]],axis=1)
    first_dataset_name = 'gtex_all'
    other_dataset_name='tcga_primtum_all_cutoffmet'
    new_dataset.columns=[first_dataset_name,other_dataset_name]
    print("1st: high cutoff applied to tcga dataset\n2nd: common genes identified with gtex dataset\n3rd: both datasets combined\n4th: High cutoff applied to gtex dataset")

    new_dataset = new_dataset[new_dataset[first_dataset_name]>=correlation_cutoff]
    dict_pd_df = pd.DataFrame(data=[[new_dataset.shape[0],new_dataset.shape[0]]],
                             index=[f'high_corr_with_{interest_gene}_both_same_genes'],
                             columns=[first_dataset_name,other_dataset_name])
    return display(HTML(dict_pd_df.to_html()))

def cutoff_applied_totcga_2_high_low(interest_gene,correlation_method,correlation_cutoff,opposing_corr_cutoff,gtex_dataset,other_dataset):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    gtex_dataset=correlation_selected_gene(gtex_dataset,interest_gene,correlation_method)
    dataset2=correlation_selected_gene(other_dataset,interest_gene,correlation_method)
    dataset2 = dataset2[dataset2>=correlation_cutoff]
    common_gene_list= gtex_dataset.index.intersection(dataset2.index)
    global new_dataset    
    new_dataset = pd.concat([gtex_dataset[common_gene_list],dataset2[common_gene_list]],axis=1)
    first_dataset_name = 'gtex_all'
    other_dataset_name='tcga_primtum_all_cutoffmet'
    new_dataset.columns=[first_dataset_name,other_dataset_name]
    print("1st: high cutoff applied to tcga dataset\n2nd: common genes identified with gtex dataset\n3rd: both datasets combined\n4th: low cutoff applied to gtex dataset")

    new_dataset = new_dataset[new_dataset[first_dataset_name]<=opposing_corr_cutoff]
    dict_pd_df = pd.DataFrame(data=[[new_dataset.shape[0],new_dataset.shape[0]]],
                             index=[f'corr_with_{interest_gene}_both_same_genes'],
                             columns=[first_dataset_name,other_dataset_name])
    return display(HTML(dict_pd_df.to_html()))

def count_genes_same_3(interest_gene,correlation_method,gtex_dataset,dataset2,dataset3):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    gtex_dataset=correlation_selected_gene(gtex_dataset,interest_gene,correlation_method)
    dataset2=correlation_selected_gene(dataset2,interest_gene,correlation_method)
    dataset3=correlation_selected_gene(dataset3,interest_gene,correlation_method)
    common_gene_list= gtex_dataset.index.intersection(dataset2.index).intersection(dataset3.index)
    global new_dataset
    new_dataset = pd.concat([gtex_dataset[common_gene_list],
                             dataset2[common_gene_list],
                             dataset3[common_gene_list]],axis=1)
    first_dataset_name = 'gtex_all'
    second_dataset_name= 'tcga_normal_all'
    third_dataset_name= 'tcga_primtum_all'
    new_dataset.columns=[first_dataset_name,second_dataset_name,third_dataset_name]
    print("1st: common genes identified for all 3 datasets\n2nd: they were counted")
    dict_pd_df = pd.DataFrame(data=[[new_dataset.shape[0],new_dataset.shape[0],new_dataset.shape[0]]],
                             index=[f'corr_with_{interest_gene}_all_same_genes'],
                             columns=[first_dataset_name,second_dataset_name,third_dataset_name])
    return display(HTML(dict_pd_df.to_html()))

def count_genes_3_no_overlap(interest_gene,correlation_method,gtex_dataset,dataset2,dataset3):
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    print('no overlap between datasets')
    dict_empty={}
    first_dataset_name = 'gtex_all'
    second_dataset_name= 'tcga_normal_all'
    third_dataset_name= 'tcga_primtum_all'
    names = [first_dataset_name,second_dataset_name,third_dataset_name]
    j=0
    for dataset in [gtex_dataset,dataset2,dataset3]:
        dataset=correlation_selected_gene(dataset,interest_gene,correlation_method)
        dict_empty[names[j]]=dataset.shape[0]
        j+=1
    dict_pd_df = pd.DataFrame(dict_empty,index=[f'corr_with_{interest_gene}'])
    return display(HTML(dict_pd_df.to_html()))

def cutoff_applied_togtex_3_high_all(interest_gene,correlation_method,correlation_cutoff,gtex_dataset,other_dataset,
                                    dataset3):
#dataset3 MUST BE tcga primtum all
#make sure to preprocess dataset
#dictionary only takes unique values, careful about this
    gtex_dataset=correlation_selected_gene(gtex_dataset,interest_gene,correlation_method)
    gtex_dataset = gtex_dataset[gtex_dataset>=correlation_cutoff]
    dataset2=correlation_selected_gene(other_dataset,interest_gene,correlation_method)
    dataset3=correlation_selected_gene(dataset3,interest_gene,correlation_method)
    common_gene_list= (gtex_dataset.index.intersection(dataset2.index)).intersection(dataset3.index)
    global new_dataset    
    new_dataset = pd.concat([gtex_dataset[common_gene_list],
                             dataset2[common_gene_list],
                            dataset3[common_gene_list]],axis=1)
    first_dataset_name = 'gtex_cutoffmet'
    other_dataset_name='tcga_normal_all'
    dataset3_name = 'tcga_primtum_all'
    new_dataset.columns=[first_dataset_name,other_dataset_name,dataset3_name]
    print("1st: high cutoff applied to gtex dataset\n2nd: common genes identified with other datasets\n3rd: all datasets combined\n4th: High cutoff applied to non-gtex datasets")
    
    new_dataset = new_dataset[(new_dataset[other_dataset_name]>=correlation_cutoff) & (new_dataset[dataset3_name]>=correlation_cutoff)]
    dict_pd_df = pd.DataFrame(data=[[new_dataset.shape[0],new_dataset.shape[0],new_dataset.shape[0]]],
                             index=[f'high_corr_with_{interest_gene}_all_same_genes'],
                             columns=[first_dataset_name,other_dataset_name,dataset3_name])
    return dict_pd_df



In [4]:
#1
count_genes_3_no_overlap('ITGA7','pearson',df_int,tcga_brca_df_normal_all,tcga_brca_df_primtum_all)


NameError: name 'df_int' is not defined

In [272]:
#2
count_genes_same_3('ITGA7','pearson',df_int,tcga_brca_df_normal_all,tcga_brca_df_primtum_all)

1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA7_all_same_genes,17313,17313,17313


In [248]:
#3
#tabulate(gtex_cutoffmet_other_correlations.iloc[1:10,:],headers='keys',tablefmt='pretty')
#display(HTML(gtex_cutoffmet_other_correlations.to_html()))
#display(HTML(tcga_normal_cutoffmet_other_corr.to_html()))
count_high_corr_genes_no_overlap('ITGA7','pearson',0.6,df_int,tcga_brca_df_normal_all)

no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


,gtex_all,tcga_normal_all
HIGHLY_correlated_with_ITGA7,319,1469


In [249]:
#4
#this one probably not very meaningful
count_high_corr_genes_with_overlap('ITGA7','pearson',0.6,df_int,tcga_brca_df_normal_all)

1st: overlapping genes identified
2nd: each dataset subset with overlapping genes
3rd: cutoff applied
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


,gtex_all,tcga_normal_all
HIGHLY_correlated_with_ITGA7,255,1332


In [40]:
#5
cutoff_applied_togtex_2_high_both('ITGA7','pearson',0.6,df_int,tcga_brca_df_normal_all)
#dataset1=correlation_selected_gene(tcga_brca_df_normal_all,'ITGA7','pearson')
#dataset2=correlation_selected_gene(tcga_brca_df_primtum_all,'ITGA7','pearson')

,gtex_cutoffmet,tcga_normal_all
high_corr_with_ITGA7_both_same_genes,247,247


In [274]:
#6
cutoff_applied_togtex_2_high_low('ITGA7','pearson',0.6,0.1,df_int,tcga_brca_df_normal_all)

insert name for other datasettcga_normal_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: low cutoff applied to non-gtex dataset


,gtex_cutoffmet,tcga_normal_all
corr_with_ITGA7_highgtex_low_other,0,0


In [46]:
#7
cutoff_applied_togtex_2_high_both('ITGA7','pearson',0.6,df_int,tcga_brca_df_primtum_all)

,gtex_cutoffmet,tcga_primtum_all
high_corr_with_ITGA7_both_same_genes,26,26


In [254]:
#8
cutoff_applied_togtex_2_high_low('ITGA7','pearson',0.6,0.1,df_int,tcga_brca_df_primtum_all)
    

insert name for other datasettcga_primtum_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: low cutoff applied to non-gtex dataset


,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA7_highgtex_low_other,69,69


In [243]:
#9
cutoff_applied_totcga_2_high_both('ITGA7','pearson',0.6,df_int,tcga_brca_df_primtum_all)

1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA7_both_same_genes,26,26


In [255]:
#10
cutoff_applied_totcga_2_high_low('ITGA7','pearson',0.6,0.1,df_int,tcga_brca_df_primtum_all)



1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA7_both_same_genes,27,27


# making the information presentable with the smaller functions

In [6]:
def make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info):
    df_int=gtex_data_processing(gtex_gct_dataset)
    tcga_data=tcga_data_processing(tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    tcga_brca_df_normal_all=tcga_data['tcga_brca_df_normal_all']
    tcga_brca_df_primtum_all=tcga_data['tcga_brca_df_primtum_all']
    #1
    count_genes_3_no_overlap(interest_gene,correlation_method,df_int,tcga_brca_df_normal_all,tcga_brca_df_primtum_all)
    #2
    count_genes_same_3(interest_gene,correlation_method,df_int,tcga_brca_df_normal_all,tcga_brca_df_primtum_all)
    #3
    print('for 1th dataset ENTER: gtex_all')
    print('for 2th dataset ENTER: tcga_normal_all')
    count_high_corr_genes_no_overlap(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all)
    #4
    #print('for 1th dataset ENTER: gtex_all')
    #print('for 2th dataset ENTER: tcga_normal_all')    
    #4 is not very meaningful
    #count_high_corr_genes_with_overlap(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all)
    #5
    print('for dataset ENTER: tcga_normal_all')    
    cutoff_applied_togtex_2_high_both(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all)
    #6
    print('for dataset ENTER: tcga_normal_all')        
    cutoff_applied_togtex_2_high_low(interest_gene,correlation_method,correlation_cutoff,opposing_corr_cutoff,df_int,tcga_brca_df_normal_all)
    #7
    print('for dataset ENTER: tcga_primtum_all')            
    cutoff_applied_togtex_2_high_both(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_primtum_all)
    #8
    print('for dataset ENTER: tcga_primtum_all')                
    cutoff_applied_togtex_2_high_low(interest_gene,correlation_method,correlation_cutoff,opposing_corr_cutoff,df_int,tcga_brca_df_primtum_all)
    #9
    cutoff_applied_totcga_2_high_both(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_primtum_all)
    #10
    cutoff_applied_totcga_2_high_low(interest_gene,correlation_method,correlation_cutoff,opposing_corr_cutoff,df_int,tcga_brca_df_primtum_all)
    return

In [7]:
def organize_gtex_tcga(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info):
    df_int=gtex_data_processing(gtex_gct_dataset)
    tcga_data=tcga_data_processing(tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    tcga_brca_df_normal_all=tcga_data['tcga_brca_df_normal_all']
    tcga_brca_df_primtum_all=tcga_data['tcga_brca_df_primtum_all']
    #3
    print('for 1th dataset ENTER: gtex_all')
    print('for 2th dataset ENTER: tcga_normal_all')
    number_3=count_high_corr_genes_no_overlap(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all)
    #4
    #print('for 1th dataset ENTER: gtex_all')
    #print('for 2th dataset ENTER: tcga_normal_all')    
    #4 is not very meaningful
    #count_high_corr_genes_with_overlap(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all)
    #5
    print('for dataset ENTER: tcga_normal_all')    
    number_5=cutoff_applied_togtex_2_high_both(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all)
    #7
    print('for dataset ENTER: tcga_primtum_all')            
    number_7=cutoff_applied_togtex_2_high_both(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_primtum_all)
    #10
    number_10=cutoff_applied_togtex_3_high_all(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all,
                                    tcga_brca_df_primtum_all)
    first_column_name='Integrin' #index
    second_column_name='GTEX_highly_correlated'
    third_column_name='%also_highly_correlated_in_TCGA_normal'
    fourth_column_name='%also_highly_correlated_in_TCGA_primtum'
    fifth_column_name='%also_highly_correlated_in_all_three'
    #ignoring first column name. setting index as integrin name (first column name)
    dict_pd_df = pd.DataFrame(data=[[number_3.iloc[0,0],
                                     str(format((number_5.iloc[0,0]/number_3.iloc[0,0])*100,'2f'))+f'({str(number_5.iloc[0,0])})',
                                     str(format((number_7.iloc[0,0]/number_3.iloc[0,0])*100,'2f'))+f'({str(number_7.iloc[0,0])})',
                                     str(format((number_10.iloc[0,0]/number_3.iloc[0,0])*100,'2f'))+f'({str(number_10.iloc[0,0])})',
                                    ]],
                              columns=[second_column_name,third_column_name,
                                      fourth_column_name,fifth_column_name],
                              index=[interest_gene])
    return display(HTML(dict_pd_df.to_html()))


In [8]:
def organize_no_overlap(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info):
    df_int=gtex_data_processing(gtex_gct_dataset)
    tcga_data=tcga_data_processing(tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    tcga_brca_df_normal_all=tcga_data['tcga_brca_df_normal_all']
    tcga_brca_df_primtum_all=tcga_data['tcga_brca_df_primtum_all']
    
    gtex_expression_mean = df_int[interest_gene].mean()
    gtex_expression_stdev= df_int[interest_gene].std()
    
    #3
    print('for 1th dataset ENTER: gtex_all')
    print('for 2th dataset ENTER: tcga_normal_all')
    number_3_tcga_normal=count_high_corr_genes_no_overlap(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_normal_all)
    #3
    print('for 1th dataset ENTER: gtex_all')
    print('for 2th dataset ENTER: tcga_primtum_all')
    number_3_tcga_primtum=count_high_corr_genes_no_overlap(interest_gene,correlation_method,correlation_cutoff,df_int,tcga_brca_df_primtum_all)
    

    first_column_name='Integrin' #index
    second_column_name='GTEX_expression_mean_&_stdev'
    third_column_name='GTEX_highly_correlated'
    fourth_column_name='highly_correlated_in_TCGA_normal'
    fifth_column_name='highly_corelated_in_TCGA_primtum'
    #ignoring first column name. setting index as integrin name (first column name)
    dict_pd_df = pd.DataFrame(data=[[f'{format(gtex_expression_mean,"0.2f")} + {format(gtex_expression_stdev,"0.2f")}',
                                     str(number_3_tcga_normal.iloc[0,0]),
                                     str(number_3_tcga_normal.iloc[0,1]),
                                     str(number_3_tcga_primtum.iloc[0,1])]],
                              columns=[second_column_name,third_column_name,
                                      fourth_column_name,fifth_column_name],
                              index=[interest_gene])
    return display(HTML(dict_pd_df.to_html()))
    
    

In [9]:
interest_gene='ITGA7'
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

# make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
#                          tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)


organize_gtex_tcga(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)

for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets


KeyboardInterrupt: 

# organize as a table (table 1)

In [58]:
integrins_list=['ITGA7','ITGB8','ITGB6','ITGA10','ITGA6','ITGA4','ITGAV','ITGA9','ITGB5','ITGA1','ITGA2','ITGA8',
'ITGB1','ITGB7','ITGA5','ITGBL1','ITGA11','ITGAL','ITGAM','ITGAX','ITGAD','ITGAE','ITGA2B','ITGB3','ITGA3',
                'ITGB4','ITGB2']

gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

# make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
#                          tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)

for interest_gene in integrins_list:
    organize_gtex_tcga(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                             tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)




for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for dataset ENTER: tcga_normal_all
insert name for other datasettcga_normal_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
for dataset ENTER: tcga_primtum_all
insert name for other datasettcga_primtum_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other datasets
3rd: all datasets combined
4th: High cutoff applied to non-gtex datasets


,GTEX_highly_correlated,%also_highly_correlated_in_TCGA_normal,%also_highly_correlated_in_TCGA_primtum,%also_highly_correlated_in_all_three
ITGA7,319,77.429467(247),8.150470(26),8.150470(26)


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for dataset ENTER: tcga_normal_all
insert name for other datasettcga_normal_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
for dataset ENTER: tcga_primtum_all
insert name for other datasettcga_primtum_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other datasets
3rd: all datasets combined
4th: High cutoff applied to non-gtex datasets


,GTEX_highly_correlated,%also_highly_correlated_in_TCGA_normal,%also_highly_correlated_in_TCGA_primtum,%also_highly_correlated_in_all_three
ITGB8,2783,40.675530(1132),0.035932(1),0.035932(1)


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for dataset ENTER: tcga_normal_all
insert name for other datasettcga_normal_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
for dataset ENTER: tcga_primtum_all
insert name for other datasettcga_primtum_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other datasets
3rd: all datasets combined
4th: High cutoff applied to non-gtex datasets


...

,GTEX_highly_correlated,%also_highly_correlated_in_TCGA_normal,%also_highly_correlated_in_TCGA_primtum,%also_highly_correlated_in_all_three
ITGA3,1431,48.916841(700),0.069881(1),0.069881(1)


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for dataset ENTER: tcga_normal_all
insert name for other datasettcga_normal_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
for dataset ENTER: tcga_primtum_all
insert name for other datasettcga_primtum_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other datasets
3rd: all datasets combined
4th: High cutoff applied to non-gtex datasets


,GTEX_highly_correlated,%also_highly_correlated_in_TCGA_normal,%also_highly_correlated_in_TCGA_primtum,%also_highly_correlated_in_all_three
ITGB4,1619,47.621989(771),0.061767(1),0.061767(1)


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for dataset ENTER: tcga_normal_all
insert name for other datasettcga_normal_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
for dataset ENTER: tcga_primtum_all
insert name for other datasettcga_primtum_all
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other dataset
3rd: both datasets combined
4th: High cutoff applied to non-gtex dataset
1st: high cutoff applied to gtex dataset
2nd: common genes identified with other datasets
3rd: all datasets combined
4th: High cutoff applied to non-gtex datasets


,GTEX_highly_correlated,%also_highly_correlated_in_TCGA_normal,%also_highly_correlated_in_TCGA_primtum,%also_highly_correlated_in_all_three
ITGB2,294,45.918367(135),40.816327(120),32.993197(97)


# organize as a table (table 3)

In [72]:
integrins_list=['ITGA7','ITGB8','ITGB6','ITGA10','ITGA6','ITGA4','ITGAV','ITGA9','ITGB5','ITGA1','ITGA2','ITGA8',
'ITGB1','ITGB7','ITGA5','ITGBL1','ITGA11','ITGAL','ITGAM','ITGAX','ITGAD','ITGAE','ITGA2B','ITGB3','ITGA3',
                'ITGB4','ITGB2']

gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

# make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
#                          tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)

for interest_gene in integrins_list:
    organize_no_overlap(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                             tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)




for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGA7,152.55 + 102.06,319,1469,119


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGB8,8.35 + 8.82,2783,2773,1


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


...

,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGBL1,12.98 + 10.20,8,2,191


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGA11,4.76 + 4.56,28,13,188


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasetgtex_all


IndexError: single positional indexer is out-of-bounds

In [73]:
#ERROR in above cell so continuing here from above cell
#'ITGA7','ITGB8','ITGB6','ITGA10','ITGA6','ITGA4','ITGAV','ITGA9','ITGB5','ITGA1','ITGA2','ITGA8',
#'ITGB1','ITGB7','ITGA5','ITGBL1','ITGA11',
integrins_list= ['ITGAL','ITGAM','ITGAX','ITGAD','ITGAE','ITGA2B','ITGB3','ITGA3',
                'ITGB4','ITGB2']

gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

# make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
#                          tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)

for interest_gene in integrins_list:
    organize_no_overlap(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                             tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)




for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGAL,4.35 + 3.46,103,285,280


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGAM,6.86 + 8.57,315,498,125


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasetgtex_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


IndexError: single positional indexer is out-of-bounds

In [ ]:
# ERROR in above cell so continuing here from above cell
#'ITGA7','ITGB8','ITGB6','ITGA10','ITGA6','ITGA4','ITGAV','ITGA9','ITGB5','ITGA1','ITGA2','ITGA8',
#'ITGB1','ITGB7','ITGA5','ITGBL1','ITGA11','ITGAL','ITGAM',
integrins_list= ['ITGAX','ITGAD','ITGAE','ITGA2B','ITGB3','ITGA3',
                'ITGB4','ITGB2']

gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

# make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
#                          tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)

for interest_gene in integrins_list:
    organize_no_overlap(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                             tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)




for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGAX,8.94 + 23.17,258,197,343


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGAD,0.19 + 0.25,4,17,10


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


...

for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGA3,42.66 + 23.39,1431,2739,1


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


,GTEX_expression_mean_&_stdev,GTEX_highly_correlated,highly_correlated_in_TCGA_normal,highly_corelated_in_TCGA_primtum
ITGB4,79.55 + 62.63,1619,2732,1


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all
for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_primtum_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_primtum_all


# doing it for every integrin

In [52]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA7'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA7,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA7_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA7_highgtex_low_other,69,69


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA7_both_same_genes,26,26


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA7_both_same_genes,27,27


In [53]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB8'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB8,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB8_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGB8_highgtex_low_other,1089,1089


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGB8_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGB8_both_same_genes,0,0


In [54]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB6'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB6,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB6_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGB6_highgtex_low_other,709,709


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGB6_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGB6_both_same_genes,0,0


In [55]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA10'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA10,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA10_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA10_highgtex_low_other,153,153


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA10_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA10_both_same_genes,0,0


In [57]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA6'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA6,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA6_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA6_highgtex_low_other,5,5


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA6_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA6_both_same_genes,0,0


In [58]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA4'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA4,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA4_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA4_highgtex_low_other,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA4_both_same_genes,24,24


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA4_both_same_genes,38,38


In [59]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGAV'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAV,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAV_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGAV_highgtex_low_other,0,0


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGAV_both_same_genes,4,4


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGAV_both_same_genes,0,0


In [61]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA9'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA9,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA9_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA9_highgtex_low_other,0,0


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA9_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA9_both_same_genes,1,1


In [62]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB5'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB5,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB5_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGB5_highgtex_low_other,6,6


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGB5_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGB5_both_same_genes,1,1


In [63]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA1'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA1,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA1_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA1_highgtex_low_other,0,0


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA1_both_same_genes,24,24


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA1_both_same_genes,37,37


In [64]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA2'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA2,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA2_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all'


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA2_highgtex_low_other,1425,1425


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA2_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA2_both_same_genes,0,0


In [65]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA8'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA8,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA8_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA8_highgtex_low_other,10,10


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA8_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA8_both_same_genes,38,38


In [5]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB1'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


NameError: name 'make_data_presentable' is not defined

In [67]:
df_int[df_int.columns[df_int.columns.str.contains("ITG")]].columns

Index(['ITGB3BP', 'ITGA10', 'ITGB1BP1', 'ITGB6', 'ITGA6', 'ITGA4', 'ITGAV',
       'ITGA9', 'ITGA9-AS1', 'ITGB5', 'ITGB5-AS1', 'ITGA1', 'ITGA2', 'ITGB8',
       'ITGA8', 'ITGB1', 'ITGB7', 'ITGA5', 'ITGA7', 'ITGBL1', 'ITGA11',
       'ITGAL', 'ITGAM', 'ITGAX', 'ITGAD', 'ITGAE', 'ITGA2B', 'ITGB3', 'ITGA3',
       'ITGB4', 'ITGB1P1', 'ITGB2', 'ITGB2-AS1', 'ITGB1BP2'],
      dtype='object', name='Description')

In [68]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB7'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB7,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB7_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGB7_highgtex_low_other,6,6


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGB7_both_same_genes,41,41


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGB7_both_same_genes,68,68


In [69]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA5'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA5,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA5_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA5_highgtex_low_other,2,2


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA5_both_same_genes,3,3


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA5_both_same_genes,24,24


In [71]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGBL1'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGBL1,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGBL1_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGBL1_highgtex_low_other,0,0


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGBL1_both_same_genes,5,5


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGBL1_both_same_genes,20,20


In [72]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA11'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA11,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA11_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_normal_all
corr_with_ITGA11_highgtex_low_other,3,3


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA11_both_same_genes,5,5


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA11_both_same_genes,30,30


In [73]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGAL'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAL,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAL_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGAL_highgtex_low_other,0,0


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGAL_both_same_genes,77,77


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGAL_both_same_genes,26,26


In [74]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGAM'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAM,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAM_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGAM_highgtex_low_other,23,23


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGAM_both_same_genes,77,77


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGAM_both_same_genes,11,11


In [76]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGAX'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAX,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAX_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGAX_highgtex_low_other,22,22


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGAX_both_same_genes,96,96


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGAX_both_same_genes,58,58


In [77]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGAD'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAD,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAD_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGAD_highgtex_low_other,0,0


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGAD_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGAD_both_same_genes,1,1


In [78]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGAE'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAE,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGAE_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGAE_highgtex_low_other,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGAE_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGAE_both_same_genes,0,0


In [79]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA2B'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA2B,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA2B_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA2B_highgtex_low_other,3,3


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA2B_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA2B_both_same_genes,0,0


In [80]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB3'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB3,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB3_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGB3_highgtex_low_other,8,8


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGB3_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGB3_both_same_genes,10,10


In [81]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA3'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA3,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGA3_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGA3_highgtex_low_other,763,763


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGA3_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGA3_both_same_genes,0,0


In [8]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB4'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB4,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB4_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGB4_highgtex_low_other,798,798


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGB4_both_same_genes,1,1


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGB4_both_same_genes,0,0


In [9]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB2'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

make_data_presentable(interest_gene,correlation_method, correlation_cutoff,opposing_corr_cutoff,gtex_gct_dataset,
                         tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)
    


no overlap between datasets


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB2,52502,19929,20248


1st: common genes identified for all 3 datasets
2nd: they were counted


,gtex_all,tcga_normal_all,tcga_primtum_all
corr_with_ITGB2_all_same_genes,17313,17313,17313


for 1th dataset ENTER: gtex_all
for 2th dataset ENTER: tcga_normal_all
no overlap between datasets
insert name for 1th datasetgtex_all
insert name for 2th datasettcga_normal_all


...

,gtex_cutoffmet,tcga_primtum_all
corr_with_ITGB2_highgtex_low_other,15,15


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: High cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
high_corr_with_ITGB2_both_same_genes,120,120


1st: high cutoff applied to tcga dataset
2nd: common genes identified with gtex dataset
3rd: both datasets combined
4th: low cutoff applied to gtex dataset


,gtex_all,tcga_primtum_all_cutoffmet
corr_with_ITGB2_both_same_genes,122,122


In [ ]:
#unique dataframes:
df_int,
tcga_brca_df_normal_all,
tcga_brca_df_primtum_all

In [4]:
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'

tcga_data=tcga_data_processing(tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info)




# big function

In [46]:
#create function
def gtex_tcga_gene_correlation_analysis(gtex_gct_dataset,interest_gene,correlation_method,correlation_cutoff,
                                       tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info,opposing_corr_cutoff):
    pearson_cutoff=correlation_cutoff
    opposing_corr_cutoff = opposing_corr_cutoff
    gtex_breast_data_all=pd.read_csv(gtex_gct_dataset,sep='\t')
    gtex_breast_data_all=gtex_breast_data_all.T.copy()
    gtex_breast_data_all.columns=gtex_breast_data_all.iloc[2]
    df_int=gtex_breast_data_all.loc[(gtex_breast_data_all.index!='id')&(gtex_breast_data_all.index!='Name')&(gtex_breast_data_all.index!='Description')].astype('float').copy()
    #correlations pearson for gtex
    pearson_ITGA7_corr_gtex=df_int.corrwith(df_int[interest_gene],method=correlation_method)
    #drop nan values
    pearson_ITGA7_corr_gtex=pearson_ITGA7_corr_gtex.dropna(axis=0,how=any)

    #gtex that meet cutoff. only focusing on positive correlations

    pearson_gtex_cutoffmet=pearson_ITGA7_corr_gtex[pearson_ITGA7_corr_gtex>=pearson_cutoff]

    #get and process tcga hiseqv2 data and sample info

    tcga_brca_df = pd.read_csv(tcga_hiseq_dataset, sep = '\t', index_col=0)
    tcga_brca_sample_info = pd.read_csv(tcga_hiseq_corresponding_sample_info,sep="\t",index_col=0)
    #drop metastatic from main sample type data frame
    tcga_brca_sample_info = tcga_brca_sample_info[~(tcga_brca_sample_info.sample_type=='Metastatic')]

    #separate normal and primtum
    tcga_brca_normal_patients = tcga_brca_sample_info[tcga_brca_sample_info.sample_type=='Solid Tissue Normal']
    tcga_brca_primtum_patients = tcga_brca_sample_info[tcga_brca_sample_info.sample_type=='Primary Tumor']

    #drop index of samples not in tcga_brca_df
    tcga_brca_normal_patients=  tcga_brca_normal_patients.drop(non_match for non_match in tcga_brca_normal_patients.index.values if non_match not in tcga_brca_df.columns)
    tcga_brca_primtum_patients =tcga_brca_primtum_patients.drop(non_match for non_match in tcga_brca_primtum_patients.index.values if non_match not in tcga_brca_df.columns) 

    #make sure both normal and tumor patients match, and drop patients not in tcga_brca_df
    matched_patient_index_list=[]
    for patient in tcga_brca_primtum_patients.patient_id.values:
        if patient in tcga_brca_normal_patients.patient_id.values and patient in [patient_id.split('-')[2] for patient_id in tcga_brca_df.columns]:
            matched_patient_index_list.append(tcga_brca_sample_info[tcga_brca_sample_info.patient_id==patient].index.values)
    matched_patient_list=tcga_brca_sample_info.loc[[x for l in matched_patient_index_list for x in l]].copy()


    #separate normal and primary tumor
    normal_matched_patient_list=matched_patient_list[matched_patient_list.sample_type=='Solid Tissue Normal']
    primtum_matched_patient_list = matched_patient_list[matched_patient_list.sample_type=='Primary Tumor']

    #matched normal and primary tumor samples
    tcga_brca_df_normal_matched=tcga_brca_df[tcga_brca_df.columns[tcga_brca_df.columns.isin(normal_matched_patient_list.index.values)]]
    tcga_brca_df_primtum_matched=tcga_brca_df[tcga_brca_df.columns[tcga_brca_df.columns.isin(primtum_matched_patient_list.index.values)]]

    #test to see if normal and primary tumor patient IDs are a match
    for i in [patient_id.split('-')[2] for patient_id in tcga_brca_df_normal_matched.columns]:
        if i not in [patient_id.split('-')[2] for patient_id in tcga_brca_df_primtum_matched.columns]:
            print('trouble')
            
    #TCGA brca normal ALL
    tcga_brca_df_normal_all = tcga_brca_df.T.loc[tcga_brca_normal_patients.index.values].copy()
    # dataframe with all TCGA primary tumor samples (not necessarily matched)
    tcga_brca_df_primtum_all= tcga_brca_df.T.loc[tcga_brca_primtum_patients.index.values].copy()
    #calculate pearson correlation 
    pearson_ITGA7_corr_tcga_brca_normal_all= tcga_brca_df_normal_all.corrwith(tcga_brca_df_normal_all[interest_gene],method=correlation_method)
    #drop nan values
    pearson_ITGA7_corr_tcga_brca_normal_all = pearson_ITGA7_corr_tcga_brca_normal_all.dropna(axis=0,how='any')
    #calculate pearson correlation 
    pearson_ITGA7_corr_tcga_brca_primtum_all= tcga_brca_df_primtum_all.corrwith(tcga_brca_df_primtum_all[interest_gene],method=correlation_method)
    #drop nan values
    pearson_ITGA7_corr_tcga_brca_primtum_all = pearson_ITGA7_corr_tcga_brca_primtum_all.dropna(axis=0,how='any')

    #pearson correlation with ITGA7, with normal/matched_primary_tumor
    pearson_ITGA7_corr_tcga_brca_normal_match=tcga_brca_df_normal_matched.T.corrwith(tcga_brca_df_normal_matched.T[interest_gene],method=correlation_method)
    #drop nan values
    pearson_ITGA7_corr_tcga_brca_normal_match=pearson_ITGA7_corr_tcga_brca_normal_match.dropna(axis=0,how='any')
    #pearson correlation with ITGA7, with normal/matched_primary_tumor
    pearson_ITGA7_corr_tcga_brca_primtum_match=tcga_brca_df_primtum_matched.T.corrwith(tcga_brca_df_primtum_matched.T[interest_gene],method=correlation_method)
    #drop nan values
    pearson_ITGA7_corr_tcga_brca_primtum_match = pearson_ITGA7_corr_tcga_brca_primtum_match.dropna(axis=0,how='any')
    
    global gtex_cutoffmet_other_correlations
    global tcga_primtum_cutoffmet_other_corr
    global gtex_cutoffmet_other_corr_triple
    global tcga_normal_cutoffmet_other_corr
    
    #gtex genes that meeet cutoff and those same genes in the tcga primary tumor samples, but cutoff only applied to gtex
    gtex_cutoffmet_other_correlations=pd.concat([pearson_gtex_cutoffmet[[gene for gene in pearson_gtex_cutoffmet.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all.index)]],
              pearson_ITGA7_corr_tcga_brca_primtum_all[[gene for gene in pearson_gtex_cutoffmet.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all.index)]]                                  ],axis=1)
    gtex_cutoffmet_other_correlations.columns=['gtex_cutoffmet','TCGA_allprimtum']

    #tcga normal that meet cutoff. only focusing on positive correlations
    pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet=pearson_ITGA7_corr_tcga_brca_normal_all[pearson_ITGA7_corr_tcga_brca_normal_all>=pearson_cutoff]    
    
    #tcga primtum that meet cutoff. only focusing on positive correlations
    pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet=pearson_ITGA7_corr_tcga_brca_primtum_all[pearson_ITGA7_corr_tcga_brca_primtum_all>=pearson_cutoff]

    #tcga primtum all genes that meet cutoff and corresponding gtex genes
    tcga_primtum_cutoffmet_other_corr=pd.concat([pearson_ITGA7_corr_gtex[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet.index)]],
              pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet.index)]]                                  ],axis=1)
    tcga_primtum_cutoffmet_other_corr.columns=['gtex_corr','TCGA_allprimtum_cutoffmet']
    
    #drop genes with duplicate values in pearson_ITGA7_corr_gtex b/c they are causing problems (doing this towards the very end)
    idx = pearson_ITGA7_corr_gtex.index.drop_duplicates(keep=False)
    pearson_ITGA7_corr_gtex = pearson_ITGA7_corr_gtex.loc[idx]

    #tcga normal all genes that meet cutoff and corresponding gtex genes
    tcga_normal_cutoffmet_other_corr=pd.concat([pearson_ITGA7_corr_gtex[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet.index)]],
                                                pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet.index)]]],axis=1)
    tcga_normal_cutoffmet_other_corr.columns=['gtex_corr','TCGA_allnormal_cutoffmet']
    
    #gtex normal all genes that meet cutoff and corresponding tcga normal AND corresponding TCGA primtum
    gtex_cutoffmet_other_corr_triple=pd.concat([pearson_gtex_cutoffmet[[gene for gene in pearson_gtex_cutoffmet.index if (gene in pearson_ITGA7_corr_tcga_brca_normal_all.index)  and (gene in pearson_ITGA7_corr_tcga_brca_primtum_all.index)]],
                                                pearson_ITGA7_corr_tcga_brca_normal_all[[gene for gene in pearson_gtex_cutoffmet.index if (gene in pearson_ITGA7_corr_tcga_brca_normal_all.index)  and (gene in pearson_ITGA7_corr_tcga_brca_primtum_all.index)]],
                                                pearson_ITGA7_corr_tcga_brca_primtum_all[[gene for gene in pearson_gtex_cutoffmet.index if (gene in pearson_ITGA7_corr_tcga_brca_normal_all.index)  and (gene in pearson_ITGA7_corr_tcga_brca_primtum_all.index)]]                                  ],axis=1)
    gtex_cutoffmet_other_corr_triple.columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum']

    
    #table for gtex cutoffmet

    total_genes_count=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations.shape[0],
                                          gtex_cutoffmet_other_correlations.shape[0]]],
                                   index=['total_genes_each'],
                 columns=['gtex_cutoffmet','TCGA_allprimtum'])
    high_corr_both=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                          gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum>=pearson_cutoff)].shape[0]]],
                                   index=['high_pos_corr_both'],
                 columns=['gtex_cutoffmet','TCGA_allprimtum'])
    #here low includes negative tcga
    high_gtex_low_tcga=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum<=opposing_corr_cutoff)].shape[0],
                                          gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum<=opposing_corr_cutoff)].shape[0]]],
                                   index=['high_gtex_low_tcga'],
                 columns=['gtex_cutoffmet','TCGA_allprimtum'])

    gtex_summary_df = pd.concat([total_genes_count,high_corr_both,high_gtex_low_tcga],axis=0)

    del total_genes_count,high_corr_both, high_gtex_low_tcga


    #table for tcga primtum cutoffmet

    total_genes_count=pd.DataFrame(data=[[tcga_primtum_cutoffmet_other_corr.shape[0],
                                          tcga_primtum_cutoffmet_other_corr.shape[0]]],
                                   index=['total_genes_each'],
                 columns=['pearson_gtex','TCGA_allprimtum_cutoffmet'])
    high_corr_both=pd.DataFrame(data=[[tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0],
                                          tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0]]],
                                   index=['high_pos_corr_both'],
                 columns=['pearson_gtex','TCGA_allprimtum_cutoffmet'])
    #here low includes negative tcga
    high_tcga_low_gtex=pd.DataFrame(data=[[tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0],
                                          tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0]]],
                                   index=['high_tcga_low_gtex'],
                 columns=['pearson_gtex','TCGA_allprimtum_cutoffmet'])

    tcga_summary_primtum_df = pd.concat([total_genes_count,high_corr_both,high_tcga_low_gtex],axis=0)

    del total_genes_count,high_corr_both, high_tcga_low_gtex
    
    #table for tcga normal cutoffmet

    total_genes_count=pd.DataFrame(data=[[tcga_normal_cutoffmet_other_corr.shape[0],
                                          tcga_normal_cutoffmet_other_corr.shape[0]]],
                                   index=['total_genes_each'],
                 columns=['pearson_gtex','TCGA_allnormal_cutoffmet'])
    high_corr_both=pd.DataFrame(data=[[tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0],
                                          tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0]]],
                                   index=['high_pos_corr_both'],
                 columns=['pearson_gtex','TCGA_allnormal_cutoffmet'])
    #here low includes negative tcga
    high_tcga_low_gtex=pd.DataFrame(data=[[tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0],
                                          tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0]]],
                                   index=['high_tcga_low_gtex'],
                 columns=['pearson_gtex','TCGA_allnormal_cutoffmet'])

    tcga_summary_normal_df = pd.concat([total_genes_count,high_corr_both,high_tcga_low_gtex],axis=0)

    del total_genes_count,high_corr_both, high_tcga_low_gtex
    
    #table for tcga normal cutoffmet

    total_genes_count=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple.shape[0],
                                          gtex_cutoffmet_other_corr_triple.shape[0],
                                         gtex_cutoffmet_other_corr_triple.shape[0]]],
                                   index=['total_genes_each'],
                 columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])
    high_corr_all=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                      gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                          gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0]]],
                                   index=['high_pos_corr_both'],
                 columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])

    high_gtex_low_tcga_normal=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0],
                                      gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0],
                                          gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0]]],
                                   index=['high_gtex_tcga_normal_low_ignore_tcga_primtum'],
                 columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])


    tcga_summary_normal_triple_df = pd.concat([total_genes_count,high_corr_all,high_gtex_low_tcga_normal],axis=0)

    del total_genes_count,high_corr_all, high_gtex_low_tcga_normal
    

    return print(gtex_summary_df,'\n',tcga_summary_primtum_df, '\n', tcga_summary_normal_df,
                '\n',tcga_summary_normal_triple_df)



    


In [14]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGA7'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

gtex_tcga_gene_correlation_analysis(gtex_gct_dataset,interest_gene,correlation_method,correlation_cutoff,
                                       tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info,opposing_corr_cutoff)



NameError: name 'gtex_tcga_gene_correlation_analysis' is not defined

In [33]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB6'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

gtex_tcga_gene_correlation_analysis(gtex_gct_dataset,interest_gene,correlation_method,correlation_cutoff,
                                       tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info,opposing_corr_cutoff)



                    gtex_cutoffmet  TCGA_allprimtum
total_genes_each              1167             1167
high_pos_corr_both               1                1
high_gtex_low_tcga             711              711 
                     pearson_gtex  TCGA_allprimtum_cutoffmet
total_genes_each               1                          1
high_pos_corr_both             1                          1
high_tcga_low_gtex             0                          0 
                     pearson_gtex  TCGA_allnormal_cutoffmet
total_genes_each            1689                      1689
high_pos_corr_both           649                       649
high_tcga_low_gtex            25                        25 
                                                ITGA7_gtex_cutoffmet  \
total_genes_each                                               1167   
high_pos_corr_both                                                1   
high_gtex_tcga_normal_low_ignore_tcga_primtum                    19   

                         

In [10]:
print(*gtex_cutoffmet_other_correlations[gtex_cutoffmet_other_correlations.TCGA_allprimtum<=0.1].index.values,sep='\n')

AGRN
C1orf159
ATAD3C
PRKCZ
MORN1
PLCH2
TP73
PHF13
GPR157
CASZ1
FBXO44
DNAJC16
ARHGEF19
TMCO4
UBXN10
PAQR7
PDIK1L
CNKSR1
GPN2
SLC9A1
SYTL1
IQCC
DCDC2B
BSDC1
KIAA1522
SFPQ
KIAA0319L
COL8A2
MEAF6
PPIEL
HIVEP3
CCDC30
FAM183A
TMEM125
C1orf210
DMAP1
ZSWIM5
FGGY
HOOK1
VAV3
KIAA1324
ST7L
CD101
NUDT17
PIAS3
PEX11B
GNRHR2
ACP6
ZNF687
CGN
TUFT1
RORC
S100A14
DCST2
DCST1
EFNA3
MSTO2P
RAB25
IGSF9
PIGM
DCAF8
USP21
NOS1AP
C1orf226
SCYL3
QSOX1
ACBD6
DDX59
TMEM9
CSRP1
ELF3
PLEKHA6
LRRN2
HHAT
FLVCR1
GPATCH2
SUSD4
NVL
TRIM11
KCNK1
FAM110C
MBOAT2
KCNS3
TTC32
KRTCAP3
C2orf16
TTC27
HEATR5B
PREPL
AHSA2
AFTPH
ATP6V1B1
FBXO41
STAMBP
SLC4A5
INO80B
C2orf68
PTCD3
PROM2
FAHD2A
SNRNP200
CNNM4
CNNM3
ANKRD39
TMEM131
C2orf15
GCC2
BCL2L11
TMEM87B
ZC3H8
CBWD2
RABL2A
TMEM177
EPB41L5
ERCC3
HS6ST1
SMPD4
SPOPL
GPD2
TTC21B
LRP2
CIR1
TTC30B
TTC30A
DNAH7
NOP58
STK36
ANKZF1
OBSL1
STK11IP
ACSL3
IRS1
ARMC9
DIS3L2
TIGD1
GIGYF2
SH3BP4
RAB17
PER2
ANKMY1
PASK
ING5
D2HGDH
EDEM1
SRGAP3
FANCD2
TATDN2
NUP210
XPC
CAPN7
ZNF860
CNOT10
EPM2AI

In [7]:
gtex_gct_dataset = 'gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2'
interest_gene = 'ITGB8'
correlation_method ='pearson'
correlation_cutoff = 0.6
tcga_hiseq_dataset = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2' 
tcga_hiseq_corresponding_sample_info = '/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix'
opposing_corr_cutoff = 0.1

gtex_tcga_gene_correlation_analysis(gtex_gct_dataset,interest_gene,correlation_method,correlation_cutoff,
                                       tcga_hiseq_dataset,tcga_hiseq_corresponding_sample_info,opposing_corr_cutoff)



NameError: name 'gtex_tcga_gene_correlation_analysis' is not defined

In [5]:
#table for tcga normal cutoffmet

total_genes_count=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple.shape[0],
                                      gtex_cutoffmet_other_corr_triple.shape[0],
                                     gtex_cutoffmet_other_corr_triple.shape[0]]],
                               index=['total_genes_each'],
             columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])
high_corr_all=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                  gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                      gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0]]],
                               index=['high_pos_corr_both'],
             columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])

high_gtex_low_tcga_normal=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0],
                                  gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0],
                                      gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0]]],
                               index=['high_gtex_tcga_normal_low_ignore_tcga_primtum'],
             columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])


tcga_summary_normal_triple_df = pd.concat([total_genes_count,high_corr_all,high_gtex_low_tcga_normal],axis=0)

del total_genes_count,high_corr_all, high_gtex_low_tcga_normal


NameError: name 'gtex_cutoffmet_other_corr_triple' is not defined

In [6]:
#procedural, not object oriented
pearson_cutoff=0.6
opposing_corr_cutoff = 0.1
gtex_breast_data_all=pd.read_csv('gene_tpm_2017-06-05_v8_breast_mammary_tissue.gct_2',sep='\t')
gtex_breast_data_all=gtex_breast_data_all.T.copy()
gtex_breast_data_all.columns=gtex_breast_data_all.iloc[2]
df_int=gtex_breast_data_all.loc[(gtex_breast_data_all.index!='id')&(gtex_breast_data_all.index!='Name')&(gtex_breast_data_all.index!='Description')].astype('float').copy()
#correlations pearson for gtex
pearson_ITGA7_corr_gtex=df_int.corrwith(df_int['ITGA7'],method='pearson')
#drop nan values
pearson_ITGA7_corr_gtex=pearson_ITGA7_corr_gtex.dropna(axis=0,how=any)

#those that meet cutoff. only focusing on positive correlations

pearson_gtex_cutoffmet=pearson_ITGA7_corr_gtex[pearson_ITGA7_corr_gtex>=pearson_cutoff]

#get and process tcga hiseqv2 data and sample info

tcga_brca_df = pd.read_csv('/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA_BRCA_HiSeqV2', sep = '\t', index_col=0)
tcga_brca_sample_info = pd.read_csv('/home/hshadman/integrins_expression/saghar_TCGA_Breast/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix',sep="\t",index_col=0)
#drop metastatic from main sample type data frame
tcga_brca_sample_info = tcga_brca_sample_info[~(tcga_brca_sample_info.sample_type=='Metastatic')]

#separate normal and primtum
tcga_brca_normal_patients = tcga_brca_sample_info[tcga_brca_sample_info.sample_type=='Solid Tissue Normal']
tcga_brca_primtum_patients = tcga_brca_sample_info[tcga_brca_sample_info.sample_type=='Primary Tumor']

#drop index of samples not in tcga_brca_df
tcga_brca_normal_patients=  tcga_brca_normal_patients.drop(non_match for non_match in tcga_brca_normal_patients.index.values if non_match not in tcga_brca_df.columns)
tcga_brca_primtum_patients =tcga_brca_primtum_patients.drop(non_match for non_match in tcga_brca_primtum_patients.index.values if non_match not in tcga_brca_df.columns) 


#make sure both normal and tumor patients match, and drop patients not in tcga_brca_df
matched_patient_index_list=[]
for patient in tcga_brca_primtum_patients.patient_id.values:
    if patient in tcga_brca_normal_patients.patient_id.values and patient in [patient_id.split('-')[2] for patient_id in tcga_brca_df.columns]:
        matched_patient_index_list.append(tcga_brca_sample_info[tcga_brca_sample_info.patient_id==patient].index.values)
matched_patient_list=tcga_brca_sample_info.loc[[x for l in matched_patient_index_list for x in l]].copy()


#separate normal and primary tumor
normal_matched_patient_list=matched_patient_list[matched_patient_list.sample_type=='Solid Tissue Normal']
primtum_matched_patient_list = matched_patient_list[matched_patient_list.sample_type=='Primary Tumor']

#matched normal and primary tumor samples
tcga_brca_df_normal_matched=tcga_brca_df[tcga_brca_df.columns[tcga_brca_df.columns.isin(normal_matched_patient_list.index.values)]]
tcga_brca_df_primtum_matched=tcga_brca_df[tcga_brca_df.columns[tcga_brca_df.columns.isin(primtum_matched_patient_list.index.values)]]

#test to see if normal and primary tumor patient IDs are a match
for i in [patient_id.split('-')[2] for patient_id in tcga_brca_df_normal_matched.columns]:
    if i not in [patient_id.split('-')[2] for patient_id in tcga_brca_df_primtum_matched.columns]:
        print('trouble')

#TCGA brca normal ALL
tcga_brca_df_normal_all = tcga_brca_df.T.loc[tcga_brca_normal_patients.index.values].copy()
# dataframe with all TCGA primary tumor samples (not necessarily matched)
tcga_brca_df_primtum_all= tcga_brca_df.T.loc[tcga_brca_primtum_patients.index.values].copy()
#calculate pearson correlation 
pearson_ITGA7_corr_tcga_brca_normal_all= tcga_brca_df_normal_all.corrwith(tcga_brca_df_normal_all['ITGA7'],method='pearson')
#drop nan values
pearson_ITGA7_corr_tcga_brca_normal_all = pearson_ITGA7_corr_tcga_brca_normal_all.dropna(axis=0,how='any')
#calculate pearson correlation 
pearson_ITGA7_corr_tcga_brca_primtum_all= tcga_brca_df_primtum_all.corrwith(tcga_brca_df_primtum_all['ITGA7'],method='pearson')
#drop nan values
pearson_ITGA7_corr_tcga_brca_primtum_all = pearson_ITGA7_corr_tcga_brca_primtum_all.dropna(axis=0,how='any')


#pearson correlation with ITGA7, with normal/matched_primary_tumor
pearson_ITGA7_corr_tcga_brca_normal_match=tcga_brca_df_normal_matched.T.corrwith(tcga_brca_df_normal_matched.T['ITGA7'],method='pearson')
#drop nan values
pearson_ITGA7_corr_tcga_brca_normal_match=pearson_ITGA7_corr_tcga_brca_normal_match.dropna(axis=0,how='any')
#pearson correlation with ITGA7, with normal/matched_primary_tumor
pearson_ITGA7_corr_tcga_brca_primtum_match=tcga_brca_df_primtum_matched.T.corrwith(tcga_brca_df_primtum_matched.T['ITGA7'],method='pearson')
#drop nan values
pearson_ITGA7_corr_tcga_brca_primtum_match = pearson_ITGA7_corr_tcga_brca_primtum_match.dropna(axis=0,how='any')

#gtex genes that meeet cutoff and those same genes in the tcga primary tumor samples, but cutoff only applied to gtex
gtex_cutoffmet_other_correlations=pd.concat([pearson_gtex_cutoffmet[[gene for gene in pearson_gtex_cutoffmet.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all.index)]],
          pearson_ITGA7_corr_tcga_brca_primtum_all[[gene for gene in pearson_gtex_cutoffmet.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all.index)]]                                  ],axis=1)
gtex_cutoffmet_other_correlations.columns=['gtex_cutoffmet','TCGA_allprimtum']

#tcga normal that meet cutoff. only focusing on positive correlations
pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet=pearson_ITGA7_corr_tcga_brca_normal_all[pearson_ITGA7_corr_tcga_brca_normal_all>=pearson_cutoff]    

#tcga primtum that meet cutoff. only focusing on positive correlations
pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet=pearson_ITGA7_corr_tcga_brca_primtum_all[pearson_ITGA7_corr_tcga_brca_primtum_all>=pearson_cutoff]


#tcga primtum all genes that meet cutoff and corresponding gtex genes
tcga_primtum_cutoffmet_other_corr=pd.concat([pearson_ITGA7_corr_gtex[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet.index)]],
          pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet.index)]]                                  ],axis=1)
tcga_primtum_cutoffmet_other_corr.columns=['gtex_corr','TCGA_allprimtum_cutoffmet']

#drop genes with duplicate values in pearson_ITGA7_corr_gtex b/c they are causing problems
idx = pearson_ITGA7_corr_gtex.index.drop_duplicates(keep=False)
pearson_ITGA7_corr_gtex = pearson_ITGA7_corr_gtex.loc[idx]

#tcga normal all genes that meet cutoff and corresponding gtex genes
tcga_normal_cutoffmet_other_corr=pd.concat([pearson_ITGA7_corr_gtex[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet.index)]],pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet[[gene for gene in pearson_ITGA7_corr_gtex.index if (gene in pearson_ITGA7_corr_tcga_brca_normal_all_cutoffmet.index)]]],axis=1)
tcga_normal_cutoffmet_other_corr.columns=['gtex_corr','TCGA_allnormal_cutoffmet']


#table for gtex cutoffmet

total_genes_count=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations.shape[0],
                                      gtex_cutoffmet_other_correlations.shape[0]]],
                               index=['total_genes_each'],
             columns=['gtex_cutoffmet','TCGA_allprimtum'])
high_corr_both=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                      gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum>=pearson_cutoff)].shape[0]]],
                               index=['high_pos_corr_both'],
             columns=['gtex_cutoffmet','TCGA_allprimtum'])
#here low includes negative tcga
high_gtex_low_tcga=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum<=opposing_corr_cutoff)].shape[0],
                                      gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum<=opposing_corr_cutoff)].shape[0]]],
                               index=['high_gtex_low_tcga'],
             columns=['gtex_cutoffmet','TCGA_allprimtum'])

gtex_summary_df = pd.concat([total_genes_count,high_corr_both,high_gtex_low_tcga],axis=0)

del total_genes_count,high_corr_both, high_gtex_low_tcga


#table for tcga primtum cutoffmet

total_genes_count=pd.DataFrame(data=[[tcga_primtum_cutoffmet_other_corr.shape[0],
                                      tcga_primtum_cutoffmet_other_corr.shape[0]]],
                               index=['total_genes_each'],
             columns=['pearson_gtex','TCGA_allprimtum_cutoffmet'])
high_corr_both=pd.DataFrame(data=[[tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0],
                                      tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0]]],
                               index=['high_pos_corr_both'],
             columns=['pearson_gtex','TCGA_allprimtum_cutoffmet'])
#here low includes negative tcga
high_tcga_low_gtex=pd.DataFrame(data=[[tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0],
                                      tcga_primtum_cutoffmet_other_corr[(tcga_primtum_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0]]],
                               index=['high_tcga_low_gtex'],
             columns=['pearson_gtex','TCGA_allprimtum_cutoffmet'])

tcga_summary_primtum_df = pd.concat([total_genes_count,high_corr_both,high_tcga_low_gtex],axis=0)

del total_genes_count,high_corr_both, high_tcga_low_gtex

#table for tcga normal cutoffmet

total_genes_count=pd.DataFrame(data=[[tcga_normal_cutoffmet_other_corr.shape[0],
                                      tcga_normal_cutoffmet_other_corr.shape[0]]],
                               index=['total_genes_each'],
             columns=['pearson_gtex','TCGA_allnormal_cutoffmet'])
high_corr_both=pd.DataFrame(data=[[tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0],
                                      tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr>=pearson_cutoff)].shape[0]]],
                               index=['high_pos_corr_both'],
             columns=['pearson_gtex','TCGA_allnormal_cutoffmet'])
#here low includes negative tcga
high_tcga_low_gtex=pd.DataFrame(data=[[tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0],
                                      tcga_normal_cutoffmet_other_corr[(tcga_normal_cutoffmet_other_corr.gtex_corr<=opposing_corr_cutoff)].shape[0]]],
                               index=['high_tcga_low_gtex'],
             columns=['pearson_gtex','TCGA_allnormal_cutoffmet'])

tcga_summary_normal_df = pd.concat([total_genes_count,high_corr_both,high_tcga_low_gtex],axis=0)


del total_genes_count,high_corr_both, high_tcga_low_gtex

#table for tcga normal cutoffmet

total_genes_count=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple.shape[0],
                                      gtex_cutoffmet_other_corr_triple.shape[0],
                                     gtex_cutoffmet_other_corr_triple.shape[0]]],
                               index=['total_genes_each'],
             columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])
high_corr_all=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                  gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                      gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX>=pearson_cutoff)  & (gtex_cutoffmet_other_corr_triple.TCGA_allprimtum>=pearson_cutoff)].shape[0]]],
                               index=['high_pos_corr_both'],
             columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])

high_gtex_low_tcga_normal=pd.DataFrame(data=[[gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0],
                                  gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0],
                                      gtex_cutoffmet_other_corr_triple[(gtex_cutoffmet_other_corr_triple.TCGA_normal_corresponding_toGTEX<=opposing_corr_cutoff)].shape[0]]],
                               index=['high_gtex_tcga_normal_low_ignore_tcga_primtum'],
             columns=['ITGA7_gtex_cutoffmet','TCGA_normal_corresponding_toGTEX','TCGA_allprimtum'])


tcga_summary_normal_triple_df = pd.concat([total_genes_count,high_corr_all,high_gtex_low_tcga_normal],axis=0)

del total_genes_count,high_corr_all, high_gtex_low_tcga_normal





NameError: name 'gtex_cutoffmet_other_corr_triple' is not defined

In [53]:
#table for gtex cutoffmet

total_genes_count=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations.shape[0],
                                      gtex_cutoffmet_other_correlations.shape[0]]],
                               index=['total_genes_each'],
             columns=['gtex_cutoffmet','TCGA_allprimtum'])
high_corr_both=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum>=pearson_cutoff)].shape[0],
                                      gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum>=pearson_cutoff)].shape[0]]],
                               index=['high_pos_corr_both'],
             columns=['gtex_cutoffmet','TCGA_allprimtum'])
#here low includes negative tcga
high_gtex_low_tcga=pd.DataFrame(data=[[gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum<=opposing_corr_cutoff)].shape[0],
                                      gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum<=opposing_corr_cutoff)].shape[0]]],
                               index=['high_gtex_low_tcga'],
             columns=['gtex_cutoffmet','TCGA_allprimtum'])

gtex_summary_df = pd.concat([total_genes_count,high_corr_both,high_gtex_low_tcga],axis=0)

del total_genes_count,high_corr_both, high_gtex_low_tcga



In [22]:
opposing_corr_cutoff = 0.1
print('table that shows TCGA tumor genes that have high correlation with ITGA7')
gtex_cutoffmet_other_correlations[(gtex_cutoffmet_other_correlations.TCGA_allprimtum<=opposing_corr_cutoff)]


table that shows TCGA tumor genes that have high correlation with ITGA7


,gtex_cutoffmet,TCGA_normal_corresponding_toGTEX,TCGA_tumor_match,TCGA_allprimtum
PGD,0.643282,0.728521,-0.054896,-0.132353
ALDH4A1,0.654665,0.779130,-0.016779,0.070603
SORT1,0.725939,0.777408,0.083804,-0.069169
PEX19,0.669230,0.837524,0.096492,-0.036779
MGST3,0.642366,0.853209,-0.068684,-0.040442
...,...,...,...,...
PDXK,0.609379,0.773694,0.069411,0.070250
SLC25A1,0.626092,0.643135,-0.066795,-0.017957
CDR1,0.633691,0.656973,0.023173,0.047713
BCAP31,0.670816,0.701794,-0.088046,-0.126229


In [28]:
# TCGA primary tumor meeting cutoff of 0.6

pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet=pearson_ITGA7_corr_tcga_brca_primtum_all[pearson_ITGA7_corr_tcga_brca_primtum_all>=pearson_cutoff]



In [31]:
pearson_ITGA7_corr_tcga_brca_primtum_all_cutoffmet.sort_values(ascending=True)

sample
IGFN1     0.600212
FIGF      0.601616
PLCXD3    0.601916
VWF       0.602454
GNG11     0.602828
            ...   
TNXB      0.726277
KCNIP2    0.734879
NPR1      0.744405
RDH5      0.753178
ITGA7     1.000000
Length: 119, dtype: float64

In [57]:
print(tcga_summary_df)

                    pearson_gtex  TCGA_allprimtum_cutoffmet
total_genes_each             111                        111
high_pos_corr_both            26                         26
high_tcga_low_gtex            27                         27
